# xgboost parameter tuning sample
grid search the best xgboost regression parameter on 1 node.

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV # gridsearchcv from sklearn
import time

## import data from blob

In [3]:
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess

block_blob_service = BlockBlobService(account_name='artificialdatastorage', account_key='/enKgvgod8Mk5cdhFlWfGrc1A/Ho73kn5LSjPUdMxS4Aj731FW0Fkmrw1qqa+Lay2uCkbVGkBO0vK3NZv6GB6g==')

container_name ='housedata'
block_blob_service.create_container(container_name) 
block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

local_path=os.path.expanduser("/data/home/shunta_ito/notebooks/sample/xgboost_grid_search/")
local_file_name ="housedata.csv"
full_path_to_file =os.path.join(local_path, local_file_name)
block_blob_service.get_blob_to_path(container_name, local_file_name, full_path_to_file)

In [4]:
housedata = pd.read_csv(local_path+local_file_name)

In [5]:
housedata.head(5)

Unnamed: 0     price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0           0  221900.0         3       1.00         1180      5650     1.0   
1           1  538000.0         3       2.25         2570      7242     2.0   
2           2  180000.0         2       1.00          770     10000     1.0   
3           3  604000.0         4       3.00         1960      5000     1.0   
4           4  510000.0         3       2.00         1680      8080     1.0   

   waterfront  view  condition  grade  sqft_above  sqft_basement  yr_built  \
0           0     0          3      7        1180              0      1955   
1           0     0          3      7        2170            400      1951   
2           0     0          3      6         770              0      1933   
3           0     0          5      7        1050            910      1965   
4           0     0          3      8        1680              0      1987   

   yr_renovated  zipcode      lat     long  sqft_living15  sqft_lot15  
0             0    98178  47.5112 -122.257           1340        5650  
1          1991    98125  47.7210 -122.319           1690        7639  
2             0    98028  47.7379 -122.233           2720        8062  
3             0    98136  47.5208 -122.393           1360        5000  
4             0    98074  47.6168 -122.045           1800        7503

In [6]:
housedata = housedata.drop("Unnamed: 0",axis=1)
housedata.head(3)
X, y = housedata.iloc[:,1:],housedata.iloc[:,0]
X = X.astype("float64")
y = y.astype("float64")
X.head(3)

bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0       3.0       1.00       1180.0    5650.0     1.0         0.0   0.0   
1       3.0       2.25       2570.0    7242.0     2.0         0.0   0.0   
2       2.0       1.00        770.0   10000.0     1.0         0.0   0.0   

   condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0        3.0    7.0      1180.0            0.0    1955.0           0.0   
1        3.0    7.0      2170.0          400.0    1951.0        1991.0   
2        3.0    6.0       770.0            0.0    1933.0           0.0   

   zipcode      lat     long  sqft_living15  sqft_lot15  
0  98178.0  47.5112 -122.257         1340.0      5650.0  
1  98125.0  47.7210 -122.319         1690.0      7639.0  
2  98028.0  47.7379 -122.233         2720.0      8062.0

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

## prepare parameter dictionary

key: parameter name demanded by model
value: parameter list

In [8]:
alphas = []
for order in range(-4,2,1):
    for num in range(1,10,1):
        alphas.append(num*(10**order))

n_ests = []
for est in range(10,50,2):
    n_ests.append(est)

l2_weights = []
for order in range(-4,2,1):
    for num in range(1,10,1):
        l2_weights.append(num*(10**order))

In [9]:
parameters = {"reg_alpha":alphas, "n_estimators":n_ests,  "reg_lambda":l2_weights}

## fit
about 7000s

In [10]:
xgb_reg = xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 30)
grid = GridSearchCV(xgb_reg, parameters,n_jobs=-1)

start = time.time()
grid.fit(X_train, y_train)
process_time = time.time() - start

print("processing time:"+str(process_time))

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


processing time:7255.039760351181


## result

In [11]:
print("best_score:"+str(grid.best_estimator_.score(X_test,y_test)))
print(grid.cv_results_["params"][grid.best_index_])

best_score:0.8764690089053895
{'n_estimators': 48, 'reg_lambda': 2, 'reg_alpha': 8}
